## Chain
The chain is the core component of the langchain package. In this section we would be doing a deep dive into 
understanding how the langchain chains work and how we can use them in our applications. 
An entire startup can be composed purely using a set of langchain chains. Without any additional overhead


In [2]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [5]:
import pandas as pd

# Data for three products and their reviews
data = {
    'Product': ['Smartphone X', 'Wireless Headphones Y', 'Smart Watch Z'],
    'Review': [
        'I love my new Smartphone X! The camera quality is amazing, and the performance is top-notch. The battery life is also impressive.',
        'Wireless Headphones Y provide excellent sound quality and are very comfortable to wear. The noise cancellation feature works like a charm.',
        'Smart Watch Z exceeded my expectations! It has a sleek design, and the fitness tracking features are incredibly accurate. The battery life is outstanding.'
    ]
}

# Creating a DataFrame
df = pd.DataFrame(data)

# Displaying the DataFrame
df.head()


,Product,Review
0,Smartphone X,I love my new Smartphone X! The camera quality...
1,Wireless Headphones Y,Wireless Headphones Y provide excellent sound ...
2,Smart Watch Z,Smart Watch Z exceeded my expectations! It has...
